In [85]:
import requests
from bs4 import BeautifulSoup
import tqdm
import sys
sys.path.append('../')
from utility.file_utility import FileUtility

base_url = 'https://www.bible.com/versions'
soup = BeautifulSoup(requests.get(base_url).content)
# collect the game ids
collect_version_links=[]
langs=[]
url_dict=dict()
for link in tqdm.tqdm(soup.select('a[target^=_self]')):
    #soup_inner = BeautifulSoup(requests.get(link).content)
    #soup.select('a[href^=https://www.bible.com/versions/*]'):
    if 'href' in link.attrs:
        if 'versions' in link.attrs['href']:
            collect_version_links.append('https://www.bible.com'+link.attrs['href'])

for url in tqdm.tqdm(collect_version_links):
    soup_inner = BeautifulSoup(requests.get(url+'.html').content)
    for lang in soup_inner.select('h2[class^=version]'):
        for x in lang.children:
            if ('<div>' in str(x)):
                langs.append(str(x).replace('<div>','').replace('</div>',''))
    for lang in soup_inner.select("a[class^='solid-button mobile-full blue']"):
        url_dict[langs[-1]]='https://www.bible.com'+lang.attrs['href']

urls=list(url_dict.keys())
urls.sort()
FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/extra_handled_bible.com.txt',['\t'.join([url.replace(' ','').replace('\t',''),url_dict[url]]) for url in urls])
    


100%|██████████| 3080/3080 [00:00<00:00, 801542.24it/s]

In [253]:
import requests
from bs4 import BeautifulSoup
import tqdm
import sys
sys.path.append('../')
from utility.file_utility import FileUtility

url_dict=dict()

#"https://find.bible/bibles/"
base_url = '/mounts/data/proj/asgari/final_proj/1000langs/config/all_bibles.html'
f=open(base_url,'r')
soup = BeautifulSoup(f)
crawled=[]
for link in tqdm.tqdm(soup.select('a')):
    if 'href' in link.attrs:
        if 'https://find.bible/bibles/' in link.attrs['href'] and 'https://find.bible/bibles/search' not in  link.attrs['href']:
               crawled.append((link.attrs['href'].split('/')[-1],'\t'.join([x.text for x in link.find_parent().find_parent().select('td')])))

#FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/biblis_extra.txt',['\t'.join(list(x)) for x in crawled])
#ids=[x.split()[1].split('bible.is/')[1].split('/')[0] for x in FileUtility.load_list('/mounts/data/proj/asgari/final_proj/1000langs/config/finalized_urls/bibleis.txt')]
#intersect=set(list(dict(crawled).keys())).intersection(ids)
#new_urls=[idx+'.txt\t'+'http://www.bible.is/'+idx+'/Matt/1' for idx in list(dict(crawled).keys()) if idx not in intersect]
#FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/biblis_extra_urls.txt',new_urls)



100%|██████████| 3422/3422 [00:00<00:00, 9405.19it/s]

In [265]:
crawled[0]

('KUDBTA', "'Auhelawa\t2008 Edition\t\tkud\t\tPapua New Guinea")

In [266]:
rows=[]
for x,y in crawled:
    a=[x]+y.split('\t')
    #if not len(a)==7:
    rows.append(a)

In [267]:
rows[0:10]

[['KUDBTA', "'Auhelawa", '2008 Edition', '', 'kud', '', 'Papua New Guinea'],
 ['KUDPNG',
  "'Auhelawa",
  "The New Testament in 'Auhelawa",
  'Yaubada Yana Walo Yemidi Vauvauna',
  'kud',
  '2008',
  'Papua New Guinea'],
 ['AIWANT',
  'Aari',
  'Aari New Testament',
  'YaʼItyop̣yā maṣhaf qedus māh̲bar. ',
  'aiw',
  '1997',
  'Ethiopia'],
 ['AIWDPI', 'Aari', 'DPI Version', '', 'aiw', '', 'Ethiopia'],
 ['AAUWBT',
  'Abau',
  'The New Testament in Abau',
  'God so Sokior-ok Iwon',
  'aau',
  '2006',
  'Papua New Guinea'],
 ['ABIANT',
  'Abidji',
  'The New Testament in Abidji',
  'Amanıé̳ Lo̳wo̳',
  'abi',
  '2001',
  'Ivory Coast'],
 ['ABIWBT',
  'Abidji',
  'The New Testament in Abidji',
  'Amanıé̳ Lo̳wo̳',
  'abi',
  '2001',
  'Ivory Coast'],
 ['ABKYAY',
  'Abkhazian',
  'The Gospels in Abkhaz',
  'Евангелие на абхазском языке',
  'abk',
  '1912',
  'Georgia'],
 ['ABNOOIA',
  'Abua',
  'New Testament in Abua',
  'Oobi Oghaaph Ilo Ajizos',
  'abn',
  '1978',
  'Nigeria'],
 ['KGRKGR'

In [256]:
['BODCTV', 'Tibetan', 'Central Tibetan Bible', 'དེང་རབས་ཀྱི་བོད་ཡིག', 'དབུས་གཙང་པར་གཞི། ', 'bod', '2015', 'China']


['BODCTV',
 'Tibetan',
 'Central Tibetan Bible',
 'དེང་རབས་ཀྱི་བོད་ཡིག',
 'དབུས་གཙང་པར་གཞི། ',
 'bod',
 '2015',
 'China']

In [268]:
import pandas as pd

In [269]:
df=pd.DataFrame(rows,columns=['code','lang','edition','name','iso','date','country','name'])

In [297]:
df=pd.read_excel('langs.xlsx')

In [298]:
cols=[idx for idx,x in enumerate(df.code.tolist()) if FileUtility.exists('/mounts/data/proj/asgari/final_proj/000_datasets/testbib/bibleis_extra/'+x+'/')]

In [303]:
df['bibleIS']=["<a href='http://www.bible.is/'"+df['code'].loc[x]+"'/Matt/1'>1</a>" if x in cols else 0 for x in range(len(df.code.tolist())) ]

In [306]:
writer = pd.ExcelWriter('with_meta.xls', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Meta data')

# Close the Pandas Excel writer and output the Excel file.
writer.save()


In [309]:
FileUtility.save_list('table.html',df.to_html().split('\n'))

In [288]:
for col in cols:
    df['bibleIS'][col]=1

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyboardInterrupt: 

In [283]:
df

,iso,lang,code,edition,Description,date,country,Available at,bibleIS
0,aai,Arifama-Miniafia,AAIWBT,Arifama-Miniafia 2009 Edition,Tur Gewasin O Baibasit Boubun,2009,Papua New Guinea,NaN,0
1,aak,Ankave,AAKWBT,Ankave 1990 Edition,Xwɨyɨá́ Gorɨxoyá Sɨŋɨṕɨrɨnɨ Yayɨyá Pɨńé,1990,Papua New Guinea,NaN,0
2,aar,Afar,AARA94,Afar New Testament,Daabur Migaq Le Kitaaba,1994,Ethiopia,NaN,0
3,aau,Abau,AAUWBT,The New Testament in Abau,God so Sokior-ok Iwon,2006,Papua New Guinea,NaN,0
4,aaz,Amarasi,AAZANT,New Testament in Amarasi,Uisneo in Kabin Ma Prenat,2014,Indonesia,NaN,0
5,aaz,Amarasi,AAZEPC,WBT Version,WBT Version NT,NaN,Indonesia,NaN,0
6,abe,Western Abnaki,ABEABE,Mark in Abenaki,Ol8jmow8gan wji malk,1830,Canada,NaN,0
7,abi,Abidji,ABIANT,The New Testament in Abidji,Amanıé̳ Lo̳wo̳,2001,Ivory Coast,NaN,0
8,abi,Abidji,ABIWBT,The New Testament in Abidji,Amanıé̳ Lo̳wo̳,2001,Ivory Coast,NaN,0
9,abk,Abkhazian,ABKYAY,The Gospels in Abkhaz,Евангелие на абхазском языке,1912,Georgia,NaN,0


In [187]:
ids=[x.split()[1].split('bible.is/')[1].split('/')[0] for x in FileUtility.load_list('/mounts/data/proj/asgari/final_proj/1000langs/config/finalized_urls/bibleis.txt')]
intersect=set(list(dict(crawled).keys())).intersection(ids)
new_urls=[idx+'.txt\t'+'http://www.bible.is/'+idx+'/Matt/1' for idx in ids if idx not in intersect]
FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/biblis_extra_urls.txt',new_urls)

In [174]:
FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/biblis_extra.txt',['\t'.join(list(x)) for x in crawled])

('KUDBTA', "'Auhelawa\t2008 Edition\t\tkud\t\tPapua New Guinea")

In [106]:

base_url = 'https://bible.cloud/inscript/content/texts/'
soup = BeautifulSoup(requests.get(base_url).content)

# collect the game ids
collect_version_links=[]
langs=[]
url_dict=dict()
for link in tqdm.tqdm(soup.select('a')):
    #soup_inner = BeautifulSoup(requests.get(link).content)
    #soup.select('a[href^=https://www.bible.com/versions/*]'):
    if 'class' in link.attrs:
        if len(link.contents)==1:
            if 'href' in link.attrs:
                url_dict[link.contents[0].replace(' ','-')]='https://bible.cloud/inscript/content/texts/'+link.attrs['href'].replace('index.html','MT1.html')

urls=list(url_dict.keys())
urls.sort()
FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/extra_handled_bible.cloud.txt',['\t'.join([url.replace(' ','').replace('\t',''),url_dict[url]]) for url in urls])



100%|██████████| 1581/1581 [00:00<00:00, 408551.21it/s]

In [108]:
urls=list(url_dict.keys())
urls.sort()
FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/extra_handled_bible.cloud.txt',['\t'.join([url.replace(' ','').replace('\t',''),url_dict[url]]) for url in urls])


In [ ]:

for url in tqdm.tqdm(collect_version_links):
    soup_inner = BeautifulSoup(requests.get(url+'.html').content)
    for lang in soup_inner.select('h2[class^=version]'):
        for x in lang.children:
            if ('<div>' in str(x)):
                langs.append(str(x).replace('<div>','').replace('</div>',''))
    for lang in soup_inner.select("a[class^='solid-button mobile-full blue']"):
        url_dict[langs[-1]]='https://www.bible.com'+lang.attrs['href']

urls=list(url_dict.keys())
urls.sort()
FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/extra_handled_bible.com.txt',['\t'.join([url.replace(' ','').replace('\t',''),url_dict[url]]) for url in urls])
    

In [192]:
seen_url=[x.split()[1] for x in FileUtility.load_list('/mounts/data/proj/asgari/final_proj/1000langs/config/finalized_urls/pngscriptures.txt')]

In [194]:
all_urls=[x.split()[1] for x in FileUtility.load_list('/mounts/data/proj/asgari/final_proj/1000langs/config/extra_handled_pngscriptures.txt')]

In [200]:
new=list(set(all_urls)-set(all_urls).intersection(seen_url))

In [201]:
all_urls=[x for x in FileUtility.load_list('/mounts/data/proj/asgari/final_proj/1000langs/config/extra_handled_pngscriptures.txt') if x.split()[1] in new]

In [203]:
FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/pngscriptures_etxra.txt',all_urls)

## Bible.com

In [ ]:
import requests
from bs4 import BeautifulSoup
import tqdm
import sys
sys.path.append('../')
from utility.file_utility import FileUtility

base_url = 'https://www.bible.com/versions'
soup = BeautifulSoup(requests.get(base_url).content)
# collect the game ids
collect_version_links=[]
langs=[]
url_dict=dict()
for link in tqdm.tqdm(soup.select('a[target^=_self]')):
    #soup_inner = BeautifulSoup(requests.get(link).content)
    #soup.select('a[href^=https://www.bible.com/versions/*]'):
    if 'href' in link.attrs:
        if 'versions' in link.attrs['href']:
            collect_version_links.append('https://www.bible.com'+link.attrs['href'])

for url in tqdm.tqdm(collect_version_links):
    soup_inner = BeautifulSoup(requests.get(url+'.html').content)
    print (soup_inner)
    #for lang in soup_inner.select('h2[class^=version]'):
    #    for x in lang.children:
    #        if ('<div>' in str(x)):
    #            langs.append(str(x).replace('<div>','').replace('</div>',''))
    #for lang in soup_inner.select("a[class^='solid-button mobile-full blue']"):
    #    url_dict[langs[-1]]='https://www.bible.com'+lang.attrs['href']

#urls=list(url_dict.keys())
#urls.sort()
#FileUtility.save_list('/mounts/data/proj/asgari/final_proj/1000langs/config/extra_handled_bible.com.txt',['\t'.join([url.replace(' ','').replace('\t',''),url_dict[url]]) for url in urls])



  0%|          | 1/1801 [00:01<36:30,  1.22s/it]

<!DOCTYPE html>
<html data-app-icon="/assets/icons/bible/200/en-df9e42178ce28600ec4049a9dc80d818.png" data-default-locale="en" data-locale="en" data-logged-in="false" dir="ltr" lang="en">
<head lang="en">
<meta content="!" name="fragment"/>
<meta content="201895780642-g4oj7hm4p3h81eg7b1di2l2l93k5gcm3.apps.googleusercontent.com" name="google-signin-client_id"/>
<meta content="profile email" name="google-signin-scope"/>
<base href="/"/>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="V5VT4lJ5Kb8qdpSrlI5zOKapNfWdVkEOE3eRUIlpvw0=" name="csrf-token"/>
<title>Young's Literal Translation 3rd Revision 1898 (YLT1898) - Download the Free Bible App.  Download now or read online. English Bible - Android, iPhone, iPad, Android tablet, Blackberry</title>
<meta content="Young's Literal Translation 3rd Revision 1898 (YLT1898) - Hundreds of versions in 900+ different languages - the Bible that goes with you anywhere. Download now or read online." name="Description"/>
<meta content


  0%|          | 2/1801 [00:02<34:19,  1.15s/it]

<!DOCTYPE html>
<html data-app-icon="/assets/icons/bible/200/en-df9e42178ce28600ec4049a9dc80d818.png" data-default-locale="en" data-locale="en" data-logged-in="false" dir="ltr" lang="en">
<head lang="en">
<meta content="!" name="fragment"/>
<meta content="201895780642-g4oj7hm4p3h81eg7b1di2l2l93k5gcm3.apps.googleusercontent.com" name="google-signin-client_id"/>
<meta content="profile email" name="google-signin-scope"/>
<base href="/"/>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="Mgin4FYQc0+96ebkS4neACed8jLDWVC5/pFHqLtwJ5Y=" name="csrf-token"/>
<title>World Messianic Bible British Edition (WMBBE) - Download the Free Bible App.  Download now or read online. English Bible - Android, iPhone, iPad, Android tablet, Blackberry</title>
<meta content="World Messianic Bible British Edition (WMBBE) - Hundreds of versions in 900+ different languages - the Bible that goes with you anywhere. Download now or read online." name="Description"/>
<meta content="/assets/icons/bibl


  0%|          | 3/1801 [00:03<34:26,  1.15s/it]

...


  0%|          | 4/1801 [00:04<34:22,  1.15s/it]

<!DOCTYPE html>
<html data-app-icon="/assets/icons/bible/200/en-df9e42178ce28600ec4049a9dc80d818.png" data-default-locale="en" data-locale="en" data-logged-in="false" dir="ltr" lang="en">
<head lang="en">
<meta content="!" name="fragment"/>
<meta content="201895780642-g4oj7hm4p3h81eg7b1di2l2l93k5gcm3.apps.googleusercontent.com" name="google-signin-client_id"/>
<meta content="profile email" name="google-signin-scope"/>
<base href="/"/>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="hybGFnrbfDg9yw3dMpC95VvySx4QH1Lq/eYHJpoNE2M=" name="csrf-token"/>
<title>World English Bible British Edition (WEBBE) - Download the Free Bible App.  Download now or read online. English Bible - Android, iPhone, iPad, Android tablet, Blackberry</title>
<meta content="World English Bible British Edition (WEBBE) - Hundreds of versions in 900+ different languages - the Bible that goes with you anywhere. Download now or read online." name="Description"/>
<meta content="/assets/icons/bible/12


  0%|          | 5/1801 [00:05<34:16,  1.14s/it]

<!DOCTYPE html>
<html data-app-icon="/assets/icons/bible/200/en-df9e42178ce28600ec4049a9dc80d818.png" data-default-locale="en" data-locale="en" data-logged-in="false" dir="ltr" lang="en">
<head lang="en">
<meta content="!" name="fragment"/>
<meta content="201895780642-g4oj7hm4p3h81eg7b1di2l2l93k5gcm3.apps.googleusercontent.com" name="google-signin-client_id"/>
<meta content="profile email" name="google-signin-scope"/>
<base href="/"/>
<meta content="authenticity_token" name="csrf-param"/>
<meta content="x/kvRrfnomWnC/Peg1W7D+xtnhopqPfDEDRyJOq95ws=" name="csrf-token"/>
<title>World English Bible (WEB) - Download the Free Bible App.  Download now or read online. English Bible - Android, iPhone, iPad, Android tablet, Blackberry</title>
<meta content="World English Bible (WEB) - Hundreds of versions in 900+ different languages - the Bible that goes with you anywhere. Download now or read online." name="Description"/>
<meta content="/assets/icons/bible/121/en-647fba4b05a37a47cb180849717279b


  0%|          | 6/1801 [00:06<33:59,  1.14s/it]